In [16]:
import sys
import getpass
import timeit
import os
import pandas as pd
import fiona
import geopandas as gpd
from arcgis import gis
username = getpass.getuser()

user = getpass.getuser()
sys.dont_write_bytecode = True

# for DataViz team members
sys.path.insert(0, '/Users/{}/Box/DataViz Projects/Utility Code'.format(user))

from utils_io import *

In [2]:
client = gis.GIS(url='https://arcgis.ad.mtc.ca.gov/portal/home/',username=username,verify_cert=False)

Enter password: ········


## Define processing functions

In [21]:
def create_gdf_from_inventory(row,client):
    print('\n----------Creating {} from data inventory----------'.format(row['Dataset_Name']))
    if row['Dataset_Source_Type'] == 'File':
        print("\nCreating geodataframe from file: ",row['Dataset_Path'])
        gdf = gpd.read_file(row['Dataset_Path'])
    else:
        print("\nCreating geodataframe from portal: ",row['Dataset_Path'])
        gdf = pull_geodata_from_argis(arcgis_data_id=row['Dataset_ID'],client=client)
    return gdf    

In [4]:
def parcel_centroid_in_poly(parcel_centroid_gdf,poly_gdf,colname):
    sj_gdf = gpd.sjoin(parcel_centroid_gdf,poly_gdf,how='left',op='within')
    sj_gdf[colname] = np.where(sj_gdf['index_right'].isnull(),0,1)
    return sj_gdf[['geo_id',colname]]

In [23]:
def point_in_parcel_poly(parcel_poly_gdf,point_gdf,colname):
    sj_gdf = gpd.sjoin(point_gdf,parcel_poly_gdf,how='right',op='within')
    sj_gdf[colname] = np.where(sj_gdf['index_left'].isnull(),0,1)
    return sj_gdf[['geo_id',colname]]

In [17]:
def batch_spatial_join(row,parcel_gdf,join_gdf):
    start = timeit.default_timer()
    print('\n----------Beginning Spatial Join Processing----------')
    if row['Spatial_Operation'] == 'Centroid Within':
        print('\nPerforming spatial join of parcel centroids within {}'.format(row['Dataset_Name']))
        parcel_centroid = parcel_gdf.set_geometry('centroid')
        sj_gdf = parcel_centroid_in_poly(parcel_centroid_gdf=parcel_centroid,
                                           poly_gdf=join_gdf,
                                           colname=row['Column_Name'])
    else:
        print('\nPerforming spatial join of {} within parcel polygons'.format(row['Dataset_Name']))
        sj_gdf = point_in_parcel_poly(parcel_poly_gdf=parcel_gdf,
                                        point_gdf=join_gdf,
                                        colname=row['Column_Name'])
    end = timeit.default_timer()
    print('\n---------Spatial Join Processing Complete and took: {}'.format(end-start))
    return sj_gdf

In [18]:
def create_flag_column(row,sj_gdf,parcel_gdf):
    print('\n----------Creating Flag Column: ',row['Column_Name'])
    col_name = row['Column_Name']
    if sj_gdf.shape[0] != parcel_gdf.shape[0]:
        parcel_gdf[col_name] = parcel_gdf['geo_id'].map(
            sj_gdf.groupby('geo_id')[col_name].first())
    else:
        parcel_gdf[col_name] = parcel_gdf['geo_id'].map(
            sj_gdf.set_index('geo_id')[col_name])

In [11]:
def batch_flag_parcel_characteristics(parcel_gdf,data_inventory_df,client):
    for index, row in data_inventory_df.iterrows():
        gdf = create_gdf_from_inventory(row,client=client)
        
        sj_gdf = batch_spatial_join(row=row,parcel_gdf=parcel_gdf,join_gdf=gdf)
        
        create_flag_column(row=row,sj_gdf=sj_gdf,parcel_gdf=parcel_gdf)      

## Read dataset inventory

In [49]:
data_inv = pd.read_csv('sb9_data_sources.csv')

## Read parcel dataset

In [13]:
parcels = gpd.read_file('data/geojson/single_family_zoning_2021.geojson')

In [14]:
#Create centroid geometry column
parcels['centroid'] = parcels.representative_point()

In [51]:
data_inv_rerun = data_inv[data_inv['Dataset_Name'] == 'Farmland Mapping and Monitoring Program']

In [52]:
batch_flag_parcel_characteristics(parcel_gdf=parcels,
                                  data_inventory_df=data_inv_rerun,
                                  client=client)


----------Creating Farmland Mapping and Monitoring Program from data inventory----------

Creating geodataframe from file:  data/geojson/ba_fmmp.geojson

----------Beginning Spatial Join Processing----------

Performing spatial join of parcel centroids within Farmland Mapping and Monitoring Program

---------Spatial Join Processing Complete and took: 33.538435862999904

----------Creating Flag Column:  fmmp


## Prepare dataset for export to geojson

In [54]:
#flag exclusion areas
cond = ((parcels['coastal_zone'] == 1) | 
        (parcels['fmmp'] == 1) | 
        (parcels['wetland'] == 1) | 
        (parcels['fire_hazard']) | 
        (parcels['hazardous_waste'] == 1) | 
        (parcels['earthquake'] == 1) | 
        (parcels['flood_hazard'] == 1) | 
        (parcels['critical_habitat'] == 1) | 
        (parcels['cced'] == 1) | 
        (parcels['historic_place'] == 1))
parcels['exclusion_area'] = np.where(cond,1,0) 

In [56]:
col_list = ['coastal_zone',
 'fmmp',
 'wetland',
 'fire_hazard',
 'hazardous_waste',
 'earthquake',
 'flood_hazard',
 'cpad',
 'critical_habitat',
 'cced',
 'historic_place',
 'exclusion_area']
parcels[col_list][parcels['exclusion_area'] == 1]

,coastal_zone,fmmp,wetland,fire_hazard,hazardous_waste,earthquake,flood_hazard,cpad,critical_habitat,cced,historic_place,exclusion_area
0,0,0,0,0,0,0,0,0,1,0,0,1
1,0,0,0,0,0,0,0,0,1,0,0,1
2,0,0,0,0,0,0,0,0,1,0,0,1
3,0,0,0,0,0,0,0,0,1,0,0,1
4,0,0,0,0,0,0,0,0,1,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...
1022788,0,0,1,0,0,0,0,0,0,0,0,1
1022858,0,0,1,0,0,0,0,0,0,0,0,1
1023148,0,0,1,0,0,0,0,1,0,0,0,1
1023230,0,0,1,0,0,0,0,1,0,0,0,1


In [58]:
out_cols = [
    'geo_id_pa',
    'jurisdiction',
    'county',
    'zn_recid',
    'zn_code',
    'zn_description',
    'zn_regional_code',
    'zn_regional_description',
    'zn_county',
    'zn_jurisdiction',
    'allows_residential',
    'editor_id',
    'edit_version',
    'urban_area',
    'coastal_zone',
    'fmmp',
    'wetland',
    'fire_hazard',
    'hazardous_waste',
    'earthquake',
    'flood_hazard',
    'cpad',
    'critical_habitat',
    'cced',
    'historic_place',
    'exclusion_area',
    'geometry'
]
parcels[out_cols].to_file('data/geojson/sb9_elligible_parcels.geojson',driver='GeoJSON')